In [3]:
import sys
import numpy

In [4]:
# enumerate binary trees of size n
# the standard form equations are B = U + V, U = z, V = B * B
# intermediate values are tabulated by a dict k -> (b_k, u_k, v_k)

idxB = 0
idxU = 1
idxV = 2

def tabulate(k, table):
    v_k = 0
    if k != 0:
        for i in range(1,k):
            v_k += table[i][idxB] * table[k-i][idxB]
    u_k = 1 if k == 1 else 0
    b_k = u_k + v_k
    table[k] = (b_k, u_k, v_k)
    return table
    
def BinaryTree(n):
    t = {}
    for k in range(n+1):
        t = tabulate(k, t)
    print t[n][idxB]
    return

for k in range(0,20):
    BinaryTree(k)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [183]:
# enumerate combinatorial structures from standard form
# supports union, cartesian product and terminal rules
# intermediate values are tabulated by a dict k -> (dict r -> r_k)

class Union:
    def __init__(self, *args): # args[0] = args[1] + args[2]
        self.Type = 'Union'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Union) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Product:
    def __init__(self, *args): # args[0] = args[1] * args[2]
        self.Type = 'Product'
        if len(args) == 3:
            self.Value = args[0]
            self.SubRule1 = args[1]
            self.SubRule2 = args[2]
        elif len(args) == 2:
            self.SubRule1 = args[0]
            self.SubRule2 = args[1]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Product) and \
            self.SubRule1 in (other.SubRule1, other.SubRule2) and \
            self.SubRule2 in (other.SubRule1, other.SubRule2) and \
            other.SubRule1 in (self.SubRule1, self.SubRule2) and \
            other.SubRule2 in (self.SubRule1, self.SubRule2)

class Terminal:
    def __init__(self, *args):
        self.Type = 'Terminal'
        if len(args) == 2:
            self.Value = args[0]
            self.Size = args[1]
        elif len(args) == 1:
            self.Size = args[0]
        else: raise Exception('Invalid parameters')
    def __eq__(self, other):
        return isinstance(other, Terminal) and self.__dict__ == other.__dict__

# note: a Cartesian product rule cannot have valuation 0
def evaluateRule(rule, k, table):
    if k not in table:
        table[k] = {}
    if k == 0:
        table[k][rule.Value] = 0
    elif isinstance(rule, Union):
        table[k][rule.Value] = table[k][rule.SubRule1] + table[k][rule.SubRule2]
    elif isinstance(rule, Product):
        table[k][rule.Value] = 0
        for i in range(1,k):
            table[k][rule.Value] += table[i][rule.SubRule1] * table[k-i][rule.SubRule2]
    elif isinstance(rule, Terminal):
        table[k][rule.Value] = 1 if rule.Size == k else 0
    else: raise Exception('Unsupported rule')
    return table

# compute the valuations of a list of rules
# implementation of algorithm on page 28, section 1.4
def computeRuleValuations(rules):
    val = {}
    for r in rules:
        if isinstance(r, Terminal):
            val[r.Value] = r.Size
        else:
            val[r.Value] = sys.maxint
    return valuate(val, rules)

# current method is to sort by decreasing valuation
# and sort all terminal rules to the front of the list
def sortRulesByValuation(rules):
    d = computeRuleValuations(rules).items()
    d_sorted = sorted(d, key=lambda x:x[1], reverse=True)
    r_sorted = []
    for (k,v) in d_sorted:
        for r in rules:
            if r.Value == k:
                if isinstance(r, Terminal):
                    r_sorted = [r] + r_sorted
                else:
                    r_sorted.append(r)
    return r_sorted
    
# helper for computeRuleValuations()
def valuate(v, rules):
    done = True
    for r in rules:
        prev = v[r.Value]
        if isinstance(r, Terminal):
            continue
        elif isinstance(r, Union):
            v[r.Value] = min(v[r.SubRule1], v[r.SubRule2])
        elif isinstance(r, Product):
            v[r.Value] = v[r.SubRule1] + v[r.SubRule2]
        else: raise Exception('Unsupported rule')
        if v[r.Value] != prev: done = False
    if done: return v
    else: return valuate(v, rules)

# rules should be sorted in order of increasing minimality
def EnumerateFromStandardForm(rules, n):
    t = {}
    for k in range(n+1):
        for r in rules:
            t = evaluateRule(r, k, t)
    return t

# test on a binary tree
def BTree(n):
    rules = [Union('B', 'U', 'V'), Product('V', 'B', 'B'), Terminal('U', 1)]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['B']

# test on a unary-binary tree
def UTree(n):
    rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    return tab[n]['U']
 
# runable code goes here
for i in range(20):
    print BTree(i)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [51]:
rules = [Union('U', 'A', 'B'), Terminal('A', 1), Union('B', 'C', 'D'), 
             Product('C', 'A', 'U'), Product('D', 'C', 'U')]
rules_sorted = sortRulesByValuation(rules)
for r in rules_sorted:
    print r.Value

A
D
C
B
U


In [213]:
# convert a symbolic equation to standard form
# example: B = Z + B * B -> [B = U + V, U = Z, V = B * B]
def convert(op, rules, v):
    op1, op2 = (op.SubRule1, op.SubRule2)
    val1, val2 = (None, None)
    evalLeft, evalRight = (False, False)
    if isinstance(op1, Product) or isinstance(op1, Union):
        for r in rules.values():
            rSub = r
            if rSub == op1:
                val1 = r.Value
                break
        if val1 == None: 
            val1 = chr(v)
            v += 1
            evalLeft = True
    elif isinstance(op1, Terminal):
        for r in rules.values():
            if r.Size == op1.Size:
                val1 = r.Value
                break
        if val1 == None:
            val1 = chr(v)
            v += 1
            op1.Value = val1
            rules[val1] = op1
            print "Adding terminal rule: " + str(op1.Value)
    else: val1 = op1
    if isinstance(op2, Product) or isinstance(op2, Union):
        for r in rules.values():
            rSub = r
            if rSub == op2:
                val2 = r.Value
                break
        if val2 == None: 
            val2 = chr(v)
            v += 1
            evalRight = True
    elif isinstance(op2, Terminal):
        for r in rules.values():
            if r.Size == op2.Size:
                val2 = r.Value
                break
        if val2 == None:
            val2 = chr(v)
            v += 1
            rules[val2] = op1
    else: val2 = op2
    op.SubRule1 = val1
    op.SubRule2 = val2
    print "Adding nonterminal rule: " + str(op.Value) + " -> " + str(op.SubRule1) + ", " + str(op.SubRule2)
    rules[op.Value] = op # rules += [op]
    if evalLeft:
        op1.Value = val1
        rules, v = convert(op1, rules, v)
    if evalRight:
        op2.Value = val2
        rules, v = convert(op2, rules, v)
    return rules, v
    
op = Union('X', Terminal(1), Union(Product(Terminal(1), 'X'), Product(Product(Terminal(1), 'X'), 'X')))
#op = Union('B', Terminal(1), Product('B', 'B'))
rules, v = convert(op, {}, 65)
for r in rules.values():
    if isinstance(r, Terminal): print "(" + str(r.Value) +", " + str(r.Size) + ")"
    else:
        print "(" + str(r.Value) +", " + str(r.SubRule1) + ", " + str(r.SubRule2) + ")"

Adding terminal rule: A
Adding nonterminal rule: X -> A, B
Adding nonterminal rule: B -> C, D
Adding nonterminal rule: C -> A, X
Adding nonterminal rule: D -> E, X
Adding nonterminal rule: E -> A, X
(A, 1)
(C, A, X)
(B, C, D)
(E, A, X)
(D, E, X)
(X, A, B)
